In [ ]:
from ortools.sat.python import cp_model
import random

# Problem size
NUM_BUCKETS = 12
NUM_COINS = 6000

# Generate sample coins
coins = []
for _ in range(NUM_COINS):
    rule = random.randint(1, 3)
    value = random.randint(1, 30)
    initial_bucket = random.randint(0, NUM_BUCKETS - 1)
    coins.append((rule, value, initial_bucket))

# Build the CP model
model = cp_model.CpModel()

# Variables: x[i][j] = 1 if coin i assigned to bucket j
x = {}  # key: (i, j), value: variable

for i, (rule, value, init) in enumerate(coins):
    if rule == 1:
        valid_buckets = [init]
    elif rule == 2:
        valid_buckets = [j for j in range(init - 2, init + 3) if 0 <= j < NUM_BUCKETS]
    elif rule == 3:
        valid_buckets = [j for j in range(init - 4, init + 5) if 0 <= j < NUM_BUCKETS]

    for j in valid_buckets:
        x[i, j] = model.NewBoolVar(f"x_{i}_{j}")

    # Each coin must be assigned to exactly one valid bucket
    model.Add(sum(x[i, j] for j in valid_buckets) == 1)

# Compute the total value per bucket
bucket_sums = []
for j in range(NUM_BUCKETS):
    sum_j = model.NewIntVar(0, NUM_COINS * 30, f"sum_bucket_{j}")
    bucket_sum_expr = []
    for i, (rule, value, init) in enumerate(coins):
        # Only include x[i, j] if it's a valid move
        if (i, j) in x:
            bucket_sum_expr.append(x[i, j] * value)
    model.Add(sum_j == sum(bucket_sum_expr))
    bucket_sums.append(sum_j)

# Calculate mean total value (integer division for now)
total_value = sum(value for _, value, _ in coins)
mean_value = total_value // NUM_BUCKETS

# Minimize total squared deviation from mean
deviations = []
for j in range(NUM_BUCKETS):
    deviation = model.NewIntVar(0, NUM_COINS * 30, f"dev_{j}")
    model.AddAbsEquality(deviation, bucket_sums[j] - mean_value)
    deviations.append(deviation)

model.Minimize(sum(deviations))  # Can use squared deviations if supported

# Solve
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 60.0  # Optional: limit solve time
status = solver.Solve(model)

# Output
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solution Found!")
    for j in range(NUM_BUCKETS):
        print(f"Bucket {j}: Total = {solver.Value(bucket_sums[j])}")
    print(f"Mean target: {mean_value}")
else:
    print("No solution found.")


coins = []
for i in range(NUM_COINS):
    serial = f"C{i:04d}"  # e.g., C0001, C0002...
    rule = random.randint(1, 3)
    value = random.randint(1, 30)
    initial_bucket = random.randint(0, NUM_BUCKETS - 1)
    coins.append((serial, rule, value, initial_bucket))

# Mapping from serial number to assigned bucket
assignments = {}

for i, (serial, rule, value, init) in enumerate(coins):
    for j in range(NUM_BUCKETS):
        if (i, j) in x and solver.BooleanValue(x[i, j]):
            assignments[serial] = j
            break  # coin is only assigned to one bucket

# Output a few
for serial in list(assignments.keys())[:10]:  # print first 10
    print(f"Coin {serial} assigned to bucket {assignments[serial]}")

import csv

with open("coin_assignments.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Serial", "Rule", "Value", "InitialBucket", "AssignedBucket"])
    for i, (serial, rule, value, init) in enumerate(coins):
        assigned_bucket = assignments.get(serial, "N/A")
        writer.writerow([serial, rule, value, init, assigned_bucket])